# Compute the Galaxy lightcurves assuming large mask

### Save metadata

Here we compute the galaxy lightcurves assuming a simple, large, mask we derive.

In [15]:
# %load /Users/obsidian/Desktop/defaults.py
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from lightkurve import KeplerTargetPixelFile
import glob

You'll need to download all the C16 data first:  
`wget -r http://quicklook.keplerscience.org/c16-galaxies/`    


Once you have that, assemble all the filenames into an iterable list.

In [3]:
fns = glob.glob('/Volumes/Truro/C16/quicklook.keplerscience.org/c16-galaxies/*.fits.gz')
len(fns)

8380

We could assemble an $n_{\mathrm{cadence}} \times n_{\mathrm{tpfs}} = (3888 \times 8380)$ array of lightcurves, then perform PCA.  

Alternatively, we could also assemble subsets of PCA grouped by channels, since we anticipate that nearby lightcurves share similar motion-induced noise characteristics.

Let's start with the "full-focal-plane" PCA first.

In [4]:
n_tpfs = len(fns)
n_cadences = 3888

# x, y centroids and lightcurve flux
x0s_array = np.zeros((n_cadences, n_tpfs))
y0s_array = np.zeros((n_cadences, n_tpfs))
lc_array = np.zeros((n_cadences, n_tpfs))
sigma_clip_array = np.zeros((n_cadences, n_tpfs))

# Channel and number of pixels in aperture and background
n_pix_ap = np.zeros(n_tpfs)
n_pix_bg = np.zeros(n_tpfs)
ch_array = np.zeros(n_tpfs)

#An array that tracks the TPFs that fail for any reason:
fail_array = np.zeros(n_tpfs, dtype=np.bool)  

In [16]:
cadn = np.arange(0, n_cadences, 1)

First we simply need to acquire the lightcurves.  We only have TPFs available, so we need to pick a mask.  We're looking for *transients* that can occur anywhere in the galaxy's disk, so I recommend a **large** mask: all pixels except the edge pixels.  We will exclude pixels that were not downloaded (nan-everywhere).

In [7]:
from astropy.utils.console import ProgressBar

In [8]:
%%time

with ProgressBar(n_tpfs, ipython_widget=True) as bar:
    for i,fn in enumerate(fns[0:n_tpfs]):
        bar.update()
        
        try:
            # Get x, y, centroids
            tpf = KeplerTargetPixelFile(fn)
            xs, ys = tpf.centroids(aperture_mask='all')
            x0s_array[:, i] = xs
            y0s_array[:, i] = ys

            # Assemble mask
            finite_mask = np.alltrue(tpf.flux==tpf.flux, axis=0)
            core_mask = tpf.pipeline_mask * False
            core_mask[1:-1, 1:-1] = True
            core_mask = core_mask & finite_mask
            bg_mask = tpf.pipeline_mask * False
            bg_mask[0, :] = True
            bg_mask[-1, :] = True
            bg_mask[:, 0] = True
            bg_mask[:, -1] = True
            bg_mask = bg_mask & finite_mask
            n_pix_ap[i] = core_mask.sum()
            n_pix_bg[i] = bg_mask.sum()

            # Compute the lightcurve
            core_lc = tpf.to_lightcurve(aperture_mask=core_mask) / core_mask.sum()
            bg_lc = tpf.to_lightcurve(aperture_mask=bg_mask) / bg_mask.sum()
            target_lc = core_lc - bg_lc.flux
            lc_array[:, i] = target_lc.flux
            
            # Identify routinely outlier flux cadences
            _, outlier_mask_sigma_clip = target_lc.flatten().remove_outliers(return_mask=True, sigma=3)
            sigma_clip_array[:, i] = outlier_mask_sigma_clip

            ch_array[i] = tpf.channel
        except:
            print("{:04d} Fail".format(i))
            fail_array[i] = True

A Jupyter Widget


CPU times: user 2h 15s, sys: 2min 40s, total: 2h 2min 55s
Wall time: 46min 26s


In [9]:
lc_array.shape

(3888, 8380)

The results are $n_{\mathrm{cadence}} \times n_{\mathrm{tpf}}$ arrays.

## Save the arrays locally for later use.

In [17]:
#! mkdir ../data/C16_raw

In [13]:
np.save('../data/C16_raw/x0s_array.npy', x0s_array)
np.save('../data/C16_raw/y0s_array.npy', y0s_array)
np.save('../data/C16_raw/lc_array.npy', lc_array)
np.save('../data/C16_raw/sigma_clip_array.npy', sigma_clip_array)
np.save('../data/C16_raw/n_pix_ap.npy', n_pix_ap)
np.save('../data/C16_raw/n_pix_bg.npy', n_pix_bg)
np.save('../data/C16_raw/ch_array.npy', ch_array)
np.save('../data/C16_raw/fail_array.npy', fail_array)

The end!